# 离线批量推理

在进行模型评估或其他任务时，通常需要对大量数据进行预测。然而，模型推理过程往往耗时较长，通过循环串行执行会增加整体时间成本，而并行执行则需要额外的开发工作。

SDK 提供了多种解决方案来应对这一场景，其中包括：

- [本地并行推理](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/batch_prediction.ipynb)：利用 SDK 内置的批量推理功能，在本地通过并行调用模型接口实现高效的批量预测。
- [数据集评估](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/dataset/batch_inference_using_dataset.ipynb)：利用 SDK 的 Dataset 模块，调用平台提供的数据集评估功能，以便快速而有效地完成任务。
- [离线批量推理](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/offline_batch_inference.ipynb)：对于时间要求不那么严格的场景，可以考虑利用平台提供的离线批量预测能力，以降低实时推理的负载压力。

本文将介绍第三种解决方案，即离线批量推理。

注意：本文的方法需要 SDK 版本 >= 0.3.4

使用前需要先将数据上传至 BOS 上，数据格式如下：

```
{"id": "1", "query": "地球的自转周期是多久？"}
{"id": "2", "query": "太阳系中最大的行星是哪颗？"}
{"id": "3", "query": "月亮是围绕地球还是围绕太阳运转的？"}
{"id": "4", "query": "水的化学式是什么？"}
{"id": "5", "query": "世界上最高的山是哪座？"}
```

之后可以通过如下方式创建一个离线批量推理任务

In [3]:
from qianfan.resources.console.data import Data

bos_name="your_bucket_name"

task = Data.create_offline_batch_inference_task(
    name="random_task_name",  # 任务名称
    descrption="task_description",  # 任务描述，可选
    # 推理模型的链接
    endpoint="https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions",
    inference_params= {   # 模型推理参数
        "temperature": 0.5,
    },
    input_bos_uri=f"bos:/{bos_name}/inference-input/",  # 输入文件在 BOS 的路径
    output_bos_uri=f"bos:/{bos_name}/inference-output/"  # 输出文件在 BOS 的路径
)

并通过如下方式获取任务的 id

In [ ]:
task_id = task['result']['taskId']
print(task_id)

根据 id 就可以获取任务的详情，得到任务的进度，等待任务完成，就可以从输出的 BOS 地址获取推理完成后的结果。

In [10]:
task_info = Data.get_offline_batch_inference_task(task_id=task_id)

status = task_info['result']['runStatus']
print(status)

Done
